# DSCI 100: Introduction to Data Science

## Tutorial 10 — Clustering: Class activity

Today, we will be looking at earthquake data from the [U.S. Geological Survey](https://www.usgs.gov/natural-hazards/earthquake-hazards/).

Each row represents seismograph measurements measured at different stations. We will be preforming a $k$-means clustering algorithm to cluster measurements based on the depth of the event (in kilometers) and magnitude of the event, a variable which characterizes the relative size.

In [ ]:
# install the necessary packages for plotting map
# comment out the line below to install, and then recomment it once it is installed (this need only be run once)
# install.packages("ggmap")

In [ ]:
# Load in necessary packages 
library(ggmap)
library(tidyverse)
library(tidyclust)
library(tidymodels)
library(broom) # importantly, don't forget broom for clustering!
options(repr.matrix.max.rows = 6)

The data set `earthquake.csv` is located in the `data` folder. Load the data set and call it `quake`.

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer
quake  

We can use the `ggmap` package to visualize the location of the earthquake activity overlaid on a map of the world.

In [ ]:
options(repr.plot.width = 15)

mapbox <- c(-179.8454, -62.3062, 179.8348, 79.6239)
register_stadiamaps(key="STADIA_MAPS_API_KEY")

my_map <- get_map(location = mapbox, source = "stadia", maptype = "stamen_toner_lite")
ggmap(my_map) +
    geom_point(data = quake, 
             aes(x = longitude, y = latitude), 
             color = "red", 
             size = 3,
             alpha = 0.5) +
    labs(x = "Longitude", y = "Latitude")

Now, let's make a scatterplot to look at the relationship between `depth` and `mag` (magnitude). 

In [ ]:
options(repr.plot.width = 7)
# your code here
fail() # No Answer - remove if you provide an answer
earthquake_plot

We will now build a `tidymodels` workflow to cluster the data. The first step is to create a `recipe` that specifies that we want to center and scale `depth` and `mag` variables in the `quake` data frame. 

*Recall that we used a `recipe` for scaling when doing classification and regression. Even though `recipe`s were originally designed for predictive modeling tasks (like classification and regression), the `tidyclust` library lets us use our familiar `tidymodels` functions for clustering too!*

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer

Now, let's use the elbow method to choose the best $k$! 💪 

(That is, the $k$ after which the WSSD improves by a *diminishing amount*.)

In [ ]:
# your code here
fail() # No Answer - remove if you provide an answer

We now have the the values for total within-cluster sum of squares for each model in a column (`total_WSSD`). Let's use it to create a line plot with points of total within-cluster sum of squares versus $K$, so that we can choose the best number of clusters to use. 

In [ ]:
kmeans_results <- kmeans_results |>
    filter(.metric == "sse_within_total") |>
    mutate(total_WSSD = mean) |>
    select(num_clusters, total_WSSD)
kmeans_results

In [ ]:
elbow_plot <- ggplot(kmeans_results, aes(x = num_clusters, y = total_WSSD)) +
  geom_point() +
  geom_line() +
  xlab("K") +
  ylab("Total within-cluster sum of squares") +
  scale_x_continuous(breaks = 1:9) +
  theme(text = element_text(size = 12))

elbow_plot

What is the optimal k? Proceed by clustering with the correct number of ks and produce a plot to go along with it. This is our final model.

In [ ]:
kmeans_best_spec <- k_means(num_clusters = 3) |>
    set_engine("stats")
kmeans_fit <- workflow() |>
    add_recipe(kmeans_recipe) |>
    add_model(kmeans_best_spec) |>
    fit(data = earthquake)

kmeans_fit

Let's revisit the scatterplot to look at the relationship between `depth` and `mag` (magnitude) across clusters.

In [ ]:
clustered_data <- kmeans_fit |>
                    augment(earthquake)
cluster_plot <- ggplot(clustered_data, aes(x = depth, y = mag, colour = .pred_cluster), size = 2) +
    geom_point() +
    labs(x = "Scaled Depth", 
         y = "Scaled Magnitude", 
         colour = "Cluster") +
    theme(text = element_text(size = 20))
cluster_plot

Now that we have our cluster assignments we can overlay the earthquake on top of the map according to their cluster.

In [ ]:
earthquake_with_cluster <- kmeans_fit |>
                    augment(quake)
ggmap(my_map) +
    geom_point(data = earthquake_with_cluster, 
             aes(x = longitude, y = latitude, colour = .pred_cluster), 
             size = 5,
             alpha = 0.5) + 
    labs(x = "Longitude", y = "Latitude", colour = "Cluster")